# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
!ollama pull llama3.2

from openai import OpenAI
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
system_prompt = "You are a technical assistant and very good at coding. \
    You are able to explain, debug as well as generate new code. \
    You are provided with a code and you need to explain what it does in a simple manner. \
    Your audience is a non-technical person. \
    Make sure you put references whenever used."
system_prompt += "You should response in a markdown format."

user_prompt = f"I am struggling with this question. Please help me out with this question: {question}"

In [8]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [9]:
print(messages)

[{'role': 'system', 'content': 'You are a technical assistant and very good at coding.     You are able to explain, debug as well as generate new code.     You are provided with a code and you need to explain what it does in a simple manner.     Your audience is a non-technical person.     Make sure you put references whenever used.You should response in a markdown format.'}, {'role': 'user', 'content': 'I am struggling with this question. Please help me out with this question: \nPlease explain what this code does and why:\nyield from {book.get("author") for book in books if book.get("author")}\n'}]


In [11]:
stream = openai.chat.completions.create(model=MODEL_LLAMA, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

**Code Explanation**
--------------------

This line of code appears to be a part of a Python generator expression, which is used to iterate over a sequence and produce a new iterable sequence.

Let's break it down:

*   `yield from`: This keyword is used to delegate the execution to another iterator. The "from" keyword indicates that we're yielding values from another source.
*   `{book.get("author") for book in books if book.get("author"}}:`

    -   **`for book in books`**: This part of the expression iterates over an iterable sequence of books (e.g., a list or dictionary).
    -   `if book.get("author")`: The `get()` method returns a value (`None` by default) for the key `"author"` if it exists in the book. If not, it is skipped.
    -   `{book.get("author")}`: For each book that passes the condition, this expression retrieves and yields the author's name from the dictionary.

**How does it work?**
----------------------

Here's a simple example of how it works:

python
import json

# Sample data in JSON format
books = [
    {"title": "Book1", "author": "AuthorA"},
    {"title": "Book2", "", "author": "AuthorB"},
    {"title": "Book3"},  # empty author field, skipped
]

for book in [book.get("author") for book in books if book.get("author")]:
    print(book)


**Output:**

AuthorA
AuthorB


As you can see, only the values from `books` that contain an `"author"` key are processed.

**Why is it used?**
--------------------

This line of code is often used in scenarios where you want to filter or transform data based on certain conditions. In this case, it's filtering out books without authors before yielding their authors.

In practice, this can be useful when working with large datasets, as it avoids unnecessary computations and memory usage.

**References:**

*   **[Yield from](https://docs.python.org/3/reference/simple_stmts.html#the-yield-statement)**: The official Python documentation on the `yield from` statement.
*   **[Generators](https://docs.python.org/3/glossary.html#term-generator)**: A concise overview of generators in Python, including generator expressions.